# 範例 : (Kaggle)房價預測
***
- 以下用房價預測資料, 觀察均值編碼的效果

# [教學目標]
- 以下用房價預測資料, 觀察均值編碼的效果

# [範例重點]
- 觀察標籤編碼與均值編碼, 在特徵數量 / 線性迴歸分數 / 線性迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 
- 觀察標籤編碼與均值編碼, 在特徵數量 / 梯度提升樹分數 / 梯度提升樹時間上, 分別有什麼影響 (In[5], Out[5], In[6], Out[6]) 

In [1]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')

train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

43 Numeric Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']



,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [3]:
# 對照組 : 標籤編碼 + 線性迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.6615606866851305
time : 0.2250227928161621 sec


In [5]:
# 均值編碼 
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['SalePrice'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['SalePrice'] , axis=1)
data.head()

,MSZoning_mean,Street_mean,Alley_mean,LotShape_mean,LandContour_mean,Utilities_mean,LotConfig_mean,LandSlope_mean,Neighborhood_mean,Condition1_mean,...,GarageType_mean,GarageFinish_mean,GarageQual_mean,GarageCond_mean,PavedDrive_mean,PoolQC_mean,Fence_mean,MiscFeature_mean,SaleType_mean,SaleCondition_mean
0,12.085891,12.025529,12.037682,11.936101,12.0227,12.024189,12.001906,12.020571,12.163647,12.042923,...,12.160783,12.172678,12.068546,12.070489,12.059901,12.022122,12.058605,12.030119,11.991068,12.005040
1,12.085891,12.025529,12.037682,11.936101,12.0227,12.024189,12.034748,12.020571,12.344180,11.818342,...,12.160783,12.172678,12.068546,12.070489,12.059901,12.022122,12.058605,12.030119,11.991068,12.005040
2,12.085891,12.025529,12.037682,12.163471,12.0227,12.024189,12.001906,12.020571,12.163647,12.042923,...,12.160783,12.172678,12.068546,12.070489,12.059901,12.022122,12.058605,12.030119,11.991068,12.005040
3,12.085891,12.025529,12.037682,12.163471,12.0227,12.024189,12.027452,12.020571,12.206664,12.042923,...,11.765651,11.818982,12.068546,12.070489,12.059901,12.022122,12.058605,12.030119,11.991068,11.788783
4,12.085891,12.025529,12.037682,12.163471,12.0227,12.024189,12.034748,12.020571,12.676003,12.042923,...,12.160783,12.172678,12.068546,12.070489,12.059901,12.022122,12.058605,12.030119,11.991068,12.005040


In [6]:
# 線性迴歸
estimator = LinearRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.7624230403716946
time : 0.038004159927368164 sec


In [7]:
# 對照組 : 標籤編碼 + 梯度提升樹
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.7765401127301261
time : 0.6520650386810303 sec


In [8]:
# 均值編碼 + 梯度提升樹
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['SalePrice'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['SalePrice'] , axis=1)
estimator = GradientBoostingRegressor()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.8050696009123234
time : 0.6190621852874756 sec


# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

In [20]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?
### A: 均值編碼在邏輯斯回歸的分數稍微高一 點，因為均值編碼是在貝葉斯的架構下，利用所要預測的應變量Y（target variable），有監督地確定最適合這個定性特徵的編碼方式。在Kaggle的數據競賽中，這也是一種常見的提高分數的手段。不過要小心容易有overfitting的情況。
[參考](https://zhuanlan.zhihu.com/p/26308272)

### 對照組 : 標籤編碼 + 邏輯斯迴歸

In [64]:
df.columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [65]:
df_temp = pd.DataFrame()
for i in df.columns:
    df_temp[i] = LabelEncoder().fit_transform(df[i])
df_temp.head()

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [66]:
df_temp.shape

(1309, 5)

In [67]:
train_X = df_temp[:train_num]
train_X.shape

(891, 5)

In [68]:
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.780004837244799
time : 0.0280001163482666 sec


### 均值編碼 + 邏輯斯迴歸
將類別使用Y的平均值來取代

In [69]:
# train set
data = pd.concat([df[:train_num], train_Y], axis = 1)
data.head()

,Name,Sex,Ticket,Cabin,Embarked,Survived
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1
4,"Allen, Mr. William Henry",male,373450,None,S,0


In [72]:
data_temp = data.groupby(['Sex'])['Survived'].mean().reset_index()
data_temp

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [42]:
data_temp = data.groupby(['Sex'])['Survived'].mean().reset_index()#計算Survived 的平均 group by Name,Sex,Ticket,Cabin,Embarked
data_temp.columns = ['Sex','Sex_mean'] # 修改欄位
data_temp

,Sex,Sex_mean
0,female,0.742038
1,male,0.188908


In [73]:
df.columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [74]:
# mean encodeing
for i in df.columns:
    mean_df = data.groupby([i])['Survived'].mean().reset_index()  
    mean_df.columns = [i, f'{i}_mean']     
    data = pd.merge(data, mean_df, on=i, how='left') #left join mean_df on column i
    data = data.drop([i] , axis=1) # remove origin column i
data = data.drop(['Survived'] , axis=1) # remove train_Y
data.head()

,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,0,0.188908,0.0,0.299854,0.336957
1,1,0.742038,1.0,1.000000,0.553571
2,1,0.742038,1.0,0.299854,0.336957
3,1,0.742038,0.5,0.500000,0.336957
4,0,0.188908,0.0,0.299854,0.336957


In [75]:
estimator = LogisticRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.020999908447265625 sec


In [82]:
data = pd.concat([df[:train_num], train_Y], axis = 1)
data.nunique()

Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
Survived      2
dtype: int64

上面運用Mean Encodeing的模型看起來overfit，這裡剔除 Name & Ticket 這兩個欄位類別特別多的，再進行模型訓練看看效果

In [83]:
# mean encodeing
for i in df.columns:
    mean_df = data.groupby([i])['Survived'].mean().reset_index()  
    mean_df.columns = [i, f'{i}_mean']     
    data = pd.merge(data, mean_df, on=i, how='left') #left join mean_df on column i
    data = data.drop([i] , axis=1) # remove origin column i
data = data.drop(['Name_mean','Ticket_mean','Survived'] , axis=1) # remove train_Y
data.head()

,Sex_mean,Cabin_mean,Embarked_mean
0,0.188908,0.299854,0.336957
1,0.742038,1.000000,0.553571
2,0.742038,0.299854,0.336957
3,0.742038,0.500000,0.336957
4,0.188908,0.299854,0.336957


In [84]:
estimator = LogisticRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 3)
score : 0.8350366889413987
time : 0.01900005340576172 sec


結果看起來好一點，比較沒有overfit的狀況。